In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
chat= ChatOpenAI(temperature=0.1,
                 streaming=True,
                 callbacks=[],)

chef_template = ChatPromptTemplate.from_messages([
    ("system","you are world-class international chef. you create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food."),
])

chef_chain = chef_template | chat
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human","{recipe}")
])

veg_chain =veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke({
    "cuisine":"indian"
})

AIMessage(content="Here's a vegetarian alternative for Chicken Tikka Masala:\n\nIngredients:\n- 500g firm tofu, cut into bite-sized pieces\n- 1 cup plain yogurt (use dairy-free yogurt for a vegan option)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves of garlic, minced\n- 1 tablespoon ginger, grated\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric powder\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 1 cup tomato puree\n- 1 cup coconut cream (or dairy-free heavy cream substitute)\n- Salt, to taste\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric powder, paprika, and salt. Mix well.\n2. Add the tofu pieces to the marinade, ensuring they are well coated. Let it marinate for at least 1 hour, or overnight in the refrigerator for better flavor.\n3. Preheat your grill or oven to medium-h

NameError: name 'ChatPromptTemplate' is not defined